# Capturing screenshots of splats and meshes

We use pyvista for visualization and screenshots of the fit splats and meshes. This allows control over camera perspectives and viewing angles while also providing solid color rendering (O3D not as nice).

In [1]:
import os
import sys
from pathlib import Path

import numpy as np
import open3d as o3d
import pyvista as pv

from collab_splats.utils.pointcloud import clean_pcd
from collab_splats.utils.visualization import (
    CAMERA_KWARGS,
    MESH_KWARGS,
    VIZ_KWARGS,
    visualize_splat,
)
from collab_splats.wrapper import Splatter, SplatterConfig

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


### Load the information of a given splat

In [2]:
base_dir = Path("/workspace/fieldwork-data/")
session_dir = base_dir / "birds/2024-02-06/SplatsSD"

# Make the configuration
splatter_config = SplatterConfig(
    file_path=session_dir / "C0043.MP4",
    method="rade-features",
    frame_proportion=0.25,  # Use 25% of the frames within the video (or default to minimum 300 frames)
)

# Initialize the Splatter class
splatter = Splatter(splatter_config)

# Call these to populate the splatter with paths (probably a better way to do this --> maybe save out config)
splatter.preprocess()
splatter.extract_features()
splatter.mesh()

transforms.json already exists at /workspace/fieldwork-data/birds/2024-02-06/environment/C0043/preproc/transforms.json
To rerun preprocessing, set overwrite=True
Output already exists for rade-features
To rerun feature extraction, set overwrite=True

Available runs:
[0] 2025-07-25_040743


### Visualize a mesh

The goal here is to make a visualization and output an image of the mesh. We use our visualization toolbox to do this! We pass in kwargs to the function, which allow us to control the plotter:
- **mesh_kwargs:** controls which attribute of the mesh is plotted ("scalars") and whether to use RGB or a color map ("rgb")
- **viz_kwargs:** controls the position of the cameras used for visualization
- **camera_kwargs:** controls the plotting of COLMAP camera frustrums 

In [5]:
import pyvista as pv

pv.start_xvfb()   # starts a virtual framebuffer automatically

mesh_fn = splatter.config["mesh_info"]["mesh"].as_posix()

plotter = visualize_splat(
    mesh=mesh_fn,
    mesh_kwargs=MESH_KWARGS,
    viz_kwargs=VIZ_KWARGS,
)

plotter.show()

/opt/conda/envs/nerfstudio/lib/python3.10/site-packages/pyvista/plotting/utilities/xvfb.py:48: PyVistaDeprecationWarning: This function is deprecated and will be removed in future version of PyVista. Use vtk-osmesa instead.
  warnings.warn(


Widget(value='<iframe src="http://localhost:37091/index.html?ui=P_0x7ba2b3575870_2&reconnect=auto" class="pyvi…

#### Plot with camera frustrums

We can also visualize the cameras that were used to fit the splat. The parameters can be controlled via the **pose_kwargs** variable.

In [4]:
# Update the kwargs for how we want them to be shown
camera_kwargs = CAMERA_KWARGS.copy()
camera_kwargs.update(
    {
        "scale": 0.002,
        "color": "red",
        "line_width": 1,
        "opacity": 0.6,
        "n_poses": 8,
    }
)

# Load the aligned poses in case we want them for visualization --> by default they are aligned to the splat
aligned_cameras = splatter.load_aligned_cameras(align_mesh=True)

plotter = visualize_splat(
    mesh=splatter.config["mesh_info"]["mesh"].as_posix(),
    aligned_cameras=aligned_cameras,
    mesh_kwargs=MESH_KWARGS,
    camera_kwargs=camera_kwargs,
    viz_kwargs=VIZ_KWARGS,
)

plotter.show()

Widget(value='<iframe src="http://localhost:37091/index.html?ui=P_0x7ba29389d3f0_1&reconnect=auto" class="pyvi…

### Visualizing a pointcloud

The pointcloud is much messier than a mesh. We first load the points and clean based on distance. There are a few other cleaning functions we can apply (e.g., density, statistical outliers), but our goal here is to minimally alter the original splat

In [ ]:
# Load the mesh transform
mesh_transform = splatter.load_mesh_transform()

# Get the pointcloud filename and load into open3d
pcd_fn = splatter.config["mesh_info"]["mesh"].parent / "splats.ply"
pcd = pcd = o3d.io.read_point_cloud(pcd_fn)

pcd, _ = clean_pcd(
    pcd, downsample=False, outlier_removal=False, distance_removal=True, reference="origin", max_distance=2.5
)

# Now map the points to the mesh
points = np.asarray(pcd.points)
points_homo = np.concatenate([points, np.ones((points.shape[0], 1))], axis=-1)
points_homo = mesh_transform["mesh_transform"] @ points_homo.T

# Move back out of homogenous coordinates
points = points_homo.T[..., :-1]

Create a pyvista object from the transformed coordinates. Plotting a pointcloud requires slightly different arguments

In [ ]:
splat = pv.PolyData(points)
splat.point_data["RGB"] = np.asarray(pcd.colors)

pcd_kwargs = MESH_KWARGS.copy()
pcd_kwargs.update(
    {
        "point_size": 0.5,
        "render_points_as_spheres": True,
        "ambient": 0.3,
        "diffuse": 0.8,
        "specular": 0.1,
    }
)

plotter = visualize_splat(
    mesh=splat,
    mesh_kwargs=pcd_kwargs,
    viz_kwargs=VIZ_KWARGS,
)

plotter.show()

### Visualize the query splat

We want to be able to show our semantic queries. We have a few forms of this -- probability maps and binary semantic masks.

In [ ]:
# Load the "tree" query
query_fn = splatter.config["mesh_info"]["mesh"].parent / "query-tree.ply"

query_kwargs = MESH_KWARGS.copy()
query_kwargs.update(
    {
        "rgb": False,
    }
)

plotter = visualize_splat(
    mesh=query_fn.as_posix(),
    mesh_kwargs=query_kwargs,
    viz_kwargs=VIZ_KWARGS,
)

plotter.show()

### Visualize a semantic segmentation query

In [ ]:
# Load the "tree" cluster segmentation
query_fn = splatter.config["mesh_info"]["mesh"].parent / "query-tree_top-cluster.ply"

plotter = visualize_splat(
    mesh=query_fn.as_posix(),
    mesh_kwargs=MESH_KWARGS,
    viz_kwargs=VIZ_KWARGS,
)

plotter.show()

### Visualize semantic features

Load the semantic features and perform dimensionality reduction to get "colors"

In [ ]:
import torch

features_fn = splatter.config["mesh_info"]["mesh"].parent / "mesh_features.pt"
features = torch.load(features_fn)

mean = features.mean(0)
n_iter = 5
with torch.no_grad():
    U, S, V = torch.pca_lowrank(features - mean, niter=n_iter)
    proj_V = V[:, :3]

low_rank = features @ proj_V
low_rank_min = torch.quantile(low_rank, 0.01, dim=0)
low_rank_max = torch.quantile(low_rank, 0.99, dim=0)

low_rank = (low_rank - low_rank_min) / (low_rank_max - low_rank_min)
low_rank = torch.clamp(low_rank, 0, 1)

Load the mesh and add a new item to point_data for the features

In [ ]:
mesh_fn = splatter.config["mesh_info"]["mesh"]
mesh = pv.read(mesh_fn)

mesh.point_data["features"] = low_rank
# splats_fn = mesh_dir / "splats.ply"

features_kwargs = MESH_KWARGS.copy()
features_kwargs.update(
    {
        "scalars": "features",
    }
)

plotter = visualize_splat(
    mesh=mesh,
    mesh_kwargs=features_kwargs,
    viz_kwargs=VIZ_KWARGS,
)

plotter.show()